In [1]:
import os
import cv2
import datetime
import json
import torch
import copy
import shutil
import ultralytics
from glob import glob
import re
import hashlib
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import math
from ultralytics import YOLO

base_path = '/home/dl.net/evezzali/Gitlab/Barcode-Localization-Benchmark/dataset/'

In [10]:
train_val_test_split = [0.6, 0.1, 0.3]

train_val_test_split = np.array(train_val_test_split)/np.sum(train_val_test_split)
th1 = train_val_test_split[0]
th2 = th1+train_val_test_split[1]

train_files = []
val_files = []
test_files = []
img_paths = glob(f'{base_path}images/*.jpg')
img_paths.extend(glob(f'{base_path}images/*.JPG'))
img_paths = list(set(img_paths))

for img_path in img_paths:
    file_name = os.path.basename(img_path)
    hash_code = int(hashlib.sha256(file_name.encode('utf-8')).hexdigest(), 16) % 10 ** 4
    if hash_code < th1 * (10 ** 4):
        train_files.append(img_path)
    elif hash_code < th2 * (10 ** 4):
        val_files.append(img_path) 
    else:
        test_files.append(img_path)
print(len(train_files), len(val_files), len(test_files))

5100 877 2616


In [3]:
glob(f'{base_path}/*.txt')

['/home/dl.net/evezzali/Gitlab/Barcode-Localization-Benchmark/dataset/train.txt',
 '/home/dl.net/evezzali/Gitlab/Barcode-Localization-Benchmark/dataset/val.txt',
 '/home/dl.net/evezzali/Gitlab/Barcode-Localization-Benchmark/dataset/test.txt']

In [4]:
#Generate txt files
string = ""

for img_path in train_files:
    file_name = os.path.basename(img_path)
    string = f'{string}{base_path}images/{file_name}' + '\n'

with open(f'{base_path}train.txt', 'w') as f:
    f.write(string)

string = ""

for img_path in test_files:
    file_name = os.path.basename(img_path)
    string = f'{string}{base_path}images/{file_name}' + '\n'

with open(f'{base_path}test.txt', 'w') as f:
    f.write(string)

string = ""

for img_path in val_files:
    file_name = os.path.basename(img_path)
    string = f'{string}{base_path}images/{file_name}' + '\n'

with open(f'{base_path}val.txt', 'w') as f:
    f.write(string)



In [15]:
label_paths = glob('./dataset/labels/*.txt')
for path in label_paths:
    f = open(path, "r")
    filtered_text = ""
    for x in f:
        splitted_line = x.split()
        if splitted_line[0] == '0':
            filtered_text += x
    f.close()
    with open('./dataset/labels2/'+os.path.basename(path), 'w') as f:
        f.write(filtered_text)

In [4]:
torch.cuda.is_available()

True

In [4]:
device = torch.device('cuda:0')

In [6]:
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)
model.to(device);

In [7]:
model.train(data=f'{base_path}data_2cl.yaml', epochs=200, imgsz=320, patience=10)

New https://pypi.org/project/ultralytics/8.0.228 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/dl.net/evezzali/Gitlab/Barcode-Localization-Benchmark/dataset/data_2cl.yaml, epochs=200, patience=10, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train47, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=N

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f58629844f0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [8]:
import shutil
model.export(format='onnx')
shutil.move('/home/dl.net/evezzali/runs/detect/train47/weights/best.onnx', './Saved Models/yolov8n_320_2cl.onnx')

Ultralytics YOLOv8.0.200 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24217MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/home/dl.net/evezzali/runs/detect/train47/weights/best.pt' with input shape (1, 3, 320, 320) BCHW and output shape(s) (1, 6, 2100) (5.9 MB)

ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success ✅ 0.3s, saved as '/home/dl.net/evezzali/runs/detect/train47/weights/best.onnx' (11.6 MB)

Export complete (0.4s)
Results saved to /home/dl.net/evezzali/runs/detect/train47/weights
Predict:         yolo predict task=detect model=/home/dl.net/evezzali/runs/detect/train47/weights/best.onnx imgsz=320  
Validate:        yolo val task=detect model=/home/dl.net/evezzali/runs/detect/train47/weights/best.onnx imgsz=320 data=/home/dl.net/evezzali/Gitlab/Barcode-Localization-Benchmark/dataset/data_2cl.yaml  
Visualize:       https://netron.app


'./Saved Models/yolov8n_320_2cl.onnx'

In [ ]:
model = YOLO('./Saved Models/yolov8s_trained.onnx', task='detect')

In [ ]:
file_path = f'{base_path}/yolo_dataset/images/Special_0099.jpg'
result = model(file_path)

In [ ]:
%matplotlib inline
img = cv2.imread(file_path)
H, W, _ = img.shape
for bbox in result[0].boxes.data.cpu().numpy():
    x0, y0, x1, y1, conf, pred = bbox
    if int(np.round(pred)) == 0:
        cv2.rectangle(img, pt1=(int(x0), int(y0)), pt2=(int(x1), int(y1)), color=(0,0,255), thickness=int((H+W)/500))
    else:
        cv2.rectangle(img, pt1=(int(x0), int(y0)), pt2=(int(x1), int(y1)), color=(255,0,0), thickness=int((H+W)/500))

plt.imshow(img[:,:,::-1])
plt.show()


In [ ]:
device = torch.device('cuda:1')
model = YOLO("./yolo.yaml");
model.to(device);

In [ ]:
model.train(data="coco.yaml", epochs=200, imgsz=640, patience=10)